## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tiksi,RU,71.6872,128.8694,-8.57,90,88,16.67,overcast clouds
1,1,Kodinsk,RU,58.6881,99.1844,-20.54,100,20,1.81,few clouds
2,2,Longyearbyen,SJ,78.2186,15.6401,8.44,78,0,12.66,clear sky
3,3,Iisalmi,FI,63.5592,27.1907,23.02,99,100,2.82,overcast clouds
4,4,Tuktoyaktuk,CA,69.4541,-133.0374,-27.40,68,20,9.22,few clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Sabang,ID,5.8933,95.3214,82.47,72,19,16.82,few clouds
7,7,Sorong,ID,-0.8833,131.2500,79.77,81,100,7.27,overcast clouds
10,10,Brokopondo,SR,4.7500,-55.1667,79.86,83,80,9.06,broken clouds
13,13,Atuona,PF,-9.8000,-139.0333,77.67,77,6,13.58,light rain
20,20,East London,ZA,-33.0153,27.9116,77.43,86,0,12.86,clear sky
26,26,Puerto Ayora,EC,-0.7393,-90.3518,87.76,84,100,4.00,overcast clouds
27,27,San Cristobal,VE,7.7669,-72.2250,77.18,38,99,6.33,overcast clouds
28,28,Cidreira,BR,-30.1811,-50.2056,78.76,69,1,18.14,clear sky
32,32,Hilo,US,19.7297,-155.0900,77.11,62,20,4.61,few clouds
35,35,Cabo San Lucas,MX,22.8909,-109.9124,83.61,35,20,8.05,few clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                206
City                   206
Country                206
Lat                    206
Lng                    206
Max Temp               206
Humidity               206
Cloudiness             206
Wind Speed             206
Current Description    206
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                206
City                   206
Country                206
Lat                    206
Lng                    206
Max Temp               206
Humidity               206
Cloudiness             206
Wind Speed             206
Current Description    206
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Sabang,ID,82.47,few clouds,5.8933,95.3214,
7,Sorong,ID,79.77,overcast clouds,-0.8833,131.2500,
10,Brokopondo,SR,79.86,broken clouds,4.7500,-55.1667,
13,Atuona,PF,77.67,light rain,-9.8000,-139.0333,
20,East London,ZA,77.43,clear sky,-33.0153,27.9116,
26,Puerto Ayora,EC,87.76,overcast clouds,-0.7393,-90.3518,
27,San Cristobal,VE,77.18,overcast clouds,7.7669,-72.2250,
28,Cidreira,BR,78.76,clear sky,-30.1811,-50.2056,
32,Hilo,US,77.11,few clouds,19.7297,-155.0900,
35,Cabo San Lucas,MX,83.61,few clouds,22.8909,-109.9124,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [33]:
hotel_df.count()

City                   206
Country                206
Max Temp               206
Current Description    206
Lat                    206
Lng                    206
Hotel Name             206
dtype: int64

In [47]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])
clean_hotel_df.count()

City                   189
Country                189
Max Temp               189
Current Description    189
Lat                    189
Lng                    189
Hotel Name             189
dtype: int64

In [48]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Sabang,ID,82.47,few clouds,5.8933,95.3214,Hotel Citra
7,Sorong,ID,79.77,overcast clouds,-0.8833,131.2500,Waigo Splash Hotel
10,Brokopondo,SR,79.86,broken clouds,4.7500,-55.1667,Matu Island
13,Atuona,PF,77.67,light rain,-9.8000,-139.0333,Villa Enata
20,East London,ZA,77.43,clear sky,-33.0153,27.9116,Tu Casa
...,...,...,...,...,...,...,...
673,Paracuru,BR,86.02,overcast clouds,-3.4100,-39.0306,Hotel Vento Brasil
674,Lobito,AO,77.02,overcast clouds,-12.3481,13.5456,Casa Rosa Hotel Residence
675,Komatipoort,ZA,76.05,clear sky,-25.4332,31.9548,Kruger View Backpackers
677,Catuday,PH,76.37,overcast clouds,16.2923,119.8062,Benjie's Kubo & Cottages


In [49]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [50]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Temperature</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [68]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 3.1), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))